# Step 2: Merging of grid and emergency calls data

**Goal of the step**: Merge the ambulance calls and the grid dataframes based on their geographical features, perform required data preparation and add the neighbourhood data to the dataframe.

**Step overview:**

1. Loading of the dataframes
2. Merging of the dataframes
3. Preparation of the data
4. Merging of the neighbourhood data

"m1" is the dataframe containing data about ambulance calls, and "m2" is the dataframe containing data about the grid.

In [ ]:
import pandas as pd
import osmnx
import geopandas as gpd
import rioxarray
import xarray
import datashader
import contextily as cx
from shapely import geometry
from shapely import wkt
import matplotlib.pyplot as plt
import seaborn
from pysal.viz import splot
from splot.esda import plot_moran
import contextily
from pysal.explore import esda
from pysal.lib import weights
from numpy.random import seed
import os
import seaborn as sns
from scipy.stats import skew
import numpy as np

## 1. Data loading

In [ ]:
# Loading of m1

m1 = pd.read_csv("data/m1.csv")
m1 = m1.drop('Unnamed: 0', axis=1)

In [ ]:
# Loading of m2

m2 = pd.read_csv("Data/grid.csv")

## 2. Data merging

In [ ]:
# Setting of the time stamp type

m1['pmeTimeStamp'] = pd.to_datetime(m1['pmeTimeStamp'])

In [ ]:
# Setting of the geometry type

m1['geometry'] = gpd.GeoSeries.from_wkt(m1['geometry'])
m1 = gpd.GeoDataFrame(m1, geometry='geometry')

In [ ]:
# Setting of the geometry type

m2['geometry'] = gpd.GeoSeries.from_wkt(m2['geometry'])
m2 = gpd.GeoDataFrame(m2, geometry='geometry')

In [ ]:
# Setting the coordinate reference system

m1.crs = 'EPSG:4326'

In [ ]:
# Setting the coordinate reference system

m1 = m1.to_crs("EPSG:28992")

In [ ]:
# Joining of the two dataframes

m1_m2 = gpd.sjoin(m1, m2, op='within')

In [ ]:
m1_m2.info()

In [ ]:
calls = m1_m2.groupby(['n_grid']).size()

In [ ]:
calls=calls.reset_index()

In [ ]:
calls.columns = ['n_grid', 'n_calls']

In [ ]:
df = pd.merge(calls, m2, on=["n_grid"])

In [ ]:
df = gpd.GeoDataFrame(df, geometry=df['geometry'])

In [ ]:
df.plot(column='n_calls', legend=True, scheme='fisher_jenks')

## 3. Data preparation

In [ ]:
df['n_calls'].hist(bins=4)

In [ ]:
sns.pairplot(df)

The number of calls appear to have significant skewness

In [ ]:
plt.boxplot(df['n_calls'])

In [ ]:
print(skew(df['n_calls'], bias=False))

In [ ]:
sns.pairplot(df)

As expected, the value is 3.9, to improve the model a transformation like log or sqrt can be performed on the feature and then the outliers can be taken into account.

The max value is too distant from the distribution, it will be removed from the dataframe.

In [ ]:
df = df[(df.n_calls != df['n_calls'].max())]

In [ ]:
print(skew(df['n_calls'], bias=False))

Now the skewness already improved significantly.

We can apply a square root transformation.

In [ ]:
df['sqrt_calls'] = np.sqrt(df['n_calls'])

In [ ]:
print(skew(df['sqrt_calls'], bias=False))

In [ ]:
df.to_csv('data/df2.csv', index=False)

In [ ]:
df

In [ ]:
# Transformation of number of calls with the sqrt function

df['sqrt_calls'] = np.sqrt(df['n_calls'])

In [ ]:
# Standardization of the root of the number of ambulance calls

df['sqrt_calls'] = (df['sqrt_calls'] - df['sqrt_calls'].mean())/df['sqrt_calls'].std()

In [ ]:
plt.boxplot(df['sqrt_calls'])

In [ ]:
print(skew(df['sqrt_calls'], bias=False))

The skewness is still significant, from the boxplot a significant outlier can be seen, it will be removed from the analysis.

In [ ]:
df = df[(df.sqrt_calls != df['sqrt_calls'].max())]

In [ ]:
print(skew(df['sqrt_calls'], bias=False))

Now the skewness is under control.

## 4. Merging of the neighbourhood data

In [ ]:
# Loading of the neighbourhood data

neigh = gpd.read_file('data/neighborhoods_2018.json').to_crs('EPSG:4326')

In [ ]:
neigh.crs = 'EPSG:28992'

In [ ]:
# Selection of the relevant features

neigh = neigh[['BU_CODE','geometry']]

In [ ]:
# Merging of the dataframes

sjoin = gpd.sjoin(df, neigh, how='left', op='intersects')

Since each cell in the grid can be partially overlapping different neighbourhoods, each cell will be assigned to the neighbourhood with the highest area overlap

In [ ]:
# Overlaying the dataframes

merged = gpd.overlay(df, neigh, how='intersection')

In [ ]:
# Calculating the overlaying area

merged['area'] = merged.geometry.area

In [ ]:
# Selecting the neighbourhood with the highest area overlap

merged = merged.sort_values(['n_grid', 'area'],ascending=False).groupby(['n_grid']).first().reset_index()

In [ ]:
# Plot of the cells assigned to each neighbourhood

fig, ax = plt.subplots(figsize=(10, 10))
neigh.plot(ax=ax, color='none', edgecolor='black')
merged.plot(ax=ax, color='none', edgecolor='r')

In [ ]:
merged.to_csv('data/df.csv', index = False)